In [1]:
## This file implements neural networks and random forest on p003ppresabsSTCC_qual.
## Due to the imbalanced data and the limited cases for class 2, we implement the over-sampling method method.
## For fully-connected neural networks, the accuracy is 96.08% for over-sampling data and 92.65% after regularization.
## For random forest, the accuracy is 98.04% for over-sampling data.
## For random forest with cross-validation, the mean accuracy is 98.53%.

In [1]:
from numpy.random import seed
seed(100)
import tensorflow
tensorflow.random.set_seed(123)

In [2]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/phage_qual/p003ppresabsSTCC_qual.csv')
df.shape

(255, 868)

In [3]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [4]:
df['pheno']

0      0
1      0
2      1
3      0
4      0
5      0
6      0
7      0
8      0
9      0
10     0
11     0
12     0
13     0
14     0
15     0
16     0
17     0
18     0
19     0
20     0
21     0
22     0
23     0
24     1
25     0
26     1
27     0
28     0
29     0
      ..
225    0
226    0
227    0
228    1
229    1
230    0
231    0
232    0
233    0
234    0
235    0
236    0
237    0
238    0
239    0
240    0
241    0
242    0
243    0
244    0
245    0
246    0
247    0
248    0
249    0
250    0
251    0
252    0
253    0
254    0
Name: pheno, Length: 255, dtype: int64

In [5]:
df['pheno'].value_counts()

0    226
1     26
2      3
Name: pheno, dtype: int64

In [6]:
df.head()

,id,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATG,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGA,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGATGTAAAGCGTAA,TTTTTTCTTTTCATAACTGTGTTGGAAATGAATTAAATTAACAGCTCTTTGTGCTTTACGGTGTGTTGC,TTTTTTCAGCATTGTCTACATTACTTAACATTCGTGTTTGTAAGTAATATTGACCGCCAATATTTAGACACTTTATAAGTATGCCATTCATCATTTTTAA,TTTTTTATCTCACCAATTTTGTAATACATCGTTCTCGTCCTCCTTGTCTTCTTCGTCCTCCTCGTTATCTTCTTCGTTTTGTAATTCATAAATTTTGTTT,TTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGATGTAAAGCGTAAA,TTTTTTAGGTACC,TTTTTGCATTCA,...,group_8664,group_8665,group_8666,group_8667,group_8836,group_8913,group_9508,ST,CC,pheno
0,107,1,1,1,1,1,0,1,1,1,...,0,0,0,0,0,0,0,5,5,0
1,109,1,1,1,1,1,0,1,1,1,...,0,0,0,0,0,0,0,8,8,0
2,115,1,1,1,0,0,0,1,1,1,...,0,0,0,0,0,0,0,5,5,1
3,120335,1,1,1,0,0,0,1,1,1,...,0,0,0,0,0,0,0,5,5,0
4,120337,1,1,1,0,0,0,1,1,1,...,0,0,0,0,0,0,0,5,5,0


In [7]:
df_clean = df.drop(columns=['id'])

In [8]:
df_clean.shape

(255, 867)

In [9]:
df_clean.head()

,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATG,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGA,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGATGTAAAGCGTAA,TTTTTTCTTTTCATAACTGTGTTGGAAATGAATTAAATTAACAGCTCTTTGTGCTTTACGGTGTGTTGC,TTTTTTCAGCATTGTCTACATTACTTAACATTCGTGTTTGTAAGTAATATTGACCGCCAATATTTAGACACTTTATAAGTATGCCATTCATCATTTTTAA,TTTTTTATCTCACCAATTTTGTAATACATCGTTCTCGTCCTCCTTGTCTTCTTCGTCCTCCTCGTTATCTTCTTCGTTTTGTAATTCATAAATTTTGTTT,TTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGATGTAAAGCGTAAA,TTTTTTAGGTACC,TTTTTGCATTCA,TTTTTCTTATTTTA,...,group_8664,group_8665,group_8666,group_8667,group_8836,group_8913,group_9508,ST,CC,pheno
0,1,1,1,1,1,0,1,1,1,1,...,0,0,0,0,0,0,0,5,5,0
1,1,1,1,1,1,0,1,1,1,1,...,0,0,0,0,0,0,0,8,8,0
2,1,1,1,0,0,0,1,1,1,1,...,0,0,0,0,0,0,0,5,5,1
3,1,1,1,0,0,0,1,1,1,1,...,0,0,0,0,0,0,0,5,5,0
4,1,1,1,0,0,0,1,1,1,1,...,0,0,0,0,0,0,0,5,5,0


In [10]:
X = df_clean.loc[:, df_clean.columns != 'pheno'].values
y = df_clean['pheno'].values
print(X.shape, y.shape)

(255, 866) (255,)


In [11]:
# over-sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_over, y_over = overS.fit_resample(X, y)
print(sorted(Counter(y_over).items()))

[(0, 226), (1, 226), (2, 226)]


Using TensorFlow backend.


In [12]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_over)

In [13]:
############# Fully-Connected Neural Network ################

In [14]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1
import numpy as np

In [15]:
#### neural network on over-sampling data
model1_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [16]:
model1_over.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [17]:
model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 474 samples, validate on 204 samples
Epoch 1/100
474/474 [==============================] - 0s 272us/step - loss: 2.1654 - accuracy: 0.5338 - val_loss: 0.7892 - val_accuracy: 0.6814
Epoch 2/100
474/474 [==============================] - 0s 88us/step - loss: 0.6484 - accuracy: 0.7426 - val_loss: 0.4788 - val_accuracy: 0.8529
Epoch 3/100
474/474 [==============================] - 0s 83us/step - loss: 0.4576 - accuracy: 0.8270 - val_loss: 0.4166 - val_accuracy: 0.8480
Epoch 4/100
474/474 [==============================] - 0s 87us/step - loss: 0.3561 - accuracy: 0.8819 - val_loss: 0.3672 - val_accuracy: 0.8627
Epoch 5/100
474/474 [==============================] - 0s 90us/step - loss: 0.3058 - accuracy: 0.9051 - val_loss: 0.3189 - val_accuracy: 0.8873
Epoch 6/100
474/474 [==============================] - 0s 90us/step - loss: 0.3046 - accuracy: 0.8987 - val_loss: 0.3371 - val_accuracy: 0.8873
Epoch 7/100
474/474 [==============================] - 0s 79us/step - loss: 0.2804 - accu

474/474 [==============================] - 0s 90us/step - loss: 0.0222 - accuracy: 1.0000 - val_loss: 0.1217 - val_accuracy: 0.9657
Epoch 58/100
474/474 [==============================] - 0s 83us/step - loss: 0.0214 - accuracy: 1.0000 - val_loss: 0.1246 - val_accuracy: 0.9657
Epoch 59/100
474/474 [==============================] - 0s 71us/step - loss: 0.0200 - accuracy: 1.0000 - val_loss: 0.1199 - val_accuracy: 0.9608
Epoch 60/100
474/474 [==============================] - 0s 65us/step - loss: 0.0211 - accuracy: 1.0000 - val_loss: 0.1145 - val_accuracy: 0.9657
Epoch 61/100
474/474 [==============================] - 0s 67us/step - loss: 0.0203 - accuracy: 1.0000 - val_loss: 0.1140 - val_accuracy: 0.9657
Epoch 62/100
474/474 [==============================] - 0s 64us/step - loss: 0.0183 - accuracy: 1.0000 - val_loss: 0.1154 - val_accuracy: 0.9608
Epoch 63/100
474/474 [==============================] - 0s 66us/step - loss: 0.0184 - accuracy: 1.0000 - val_loss: 0.1015 - val_accuracy: 0.970

In [18]:
acc_test_over = model1_over.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over*100))

204/204 [==============================] - 0s 62us/step
over-sampling test accuracy: 96.08%


In [19]:
#### add regularizor and dropout
model1_over2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],), activity_regularizer=l1(0.001)),
    Dense(3, activation='softmax'),
    Dropout(0.2, ),
])

In [20]:
model1_over2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [21]:
model1_over2.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 474 samples, validate on 204 samples
Epoch 1/100
474/474 [==============================] - 0s 378us/step - loss: 9.8557 - accuracy: 0.2869 - val_loss: 5.7160 - val_accuracy: 0.4804
Epoch 2/100
474/474 [==============================] - 0s 98us/step - loss: 6.5014 - accuracy: 0.4705 - val_loss: 2.3208 - val_accuracy: 0.6422
Epoch 3/100
474/474 [==============================] - 0s 90us/step - loss: 4.8621 - accuracy: 0.5907 - val_loss: 1.7799 - val_accuracy: 0.7206
Epoch 4/100
474/474 [==============================] - 0s 90us/step - loss: 3.9574 - accuracy: 0.6456 - val_loss: 1.4958 - val_accuracy: 0.7647
Epoch 5/100
474/474 [==============================] - 0s 94us/step - loss: 4.1307 - accuracy: 0.6371 - val_loss: 1.4853 - val_accuracy: 0.7206
Epoch 6/100
474/474 [==============================] - 0s 92us/step - loss: 3.8210 - accuracy: 0.6477 - val_loss: 1.2382 - val_accuracy: 0.8088
Epoch 7/100
474/474 [==============================] - 0s 88us/step - loss: 3.5170 - accu

474/474 [==============================] - 0s 95us/step - loss: 1.6512 - accuracy: 0.7932 - val_loss: 1.0427 - val_accuracy: 0.9314
Epoch 58/100
474/474 [==============================] - 0s 88us/step - loss: 1.4295 - accuracy: 0.8291 - val_loss: 0.9532 - val_accuracy: 0.9314
Epoch 59/100
474/474 [==============================] - 0s 85us/step - loss: 1.5409 - accuracy: 0.8059 - val_loss: 0.9954 - val_accuracy: 0.9363
Epoch 60/100
474/474 [==============================] - 0s 75us/step - loss: 1.4951 - accuracy: 0.8059 - val_loss: 0.9975 - val_accuracy: 0.9314
Epoch 61/100
474/474 [==============================] - 0s 74us/step - loss: 1.4787 - accuracy: 0.7911 - val_loss: 1.0436 - val_accuracy: 0.9216
Epoch 62/100
474/474 [==============================] - 0s 77us/step - loss: 1.5320 - accuracy: 0.7869 - val_loss: 1.0436 - val_accuracy: 0.9265
Epoch 63/100
474/474 [==============================] - 0s 81us/step - loss: 1.7175 - accuracy: 0.7785 - val_loss: 1.0867 - val_accuracy: 0.926

In [22]:
acc_test_over2 = model1_over2.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over2*100))

204/204 [==============================] - 0s 73us/step
over-sampling test accuracy: 92.65%


In [23]:
############## Random Forest ##############

In [24]:
###### random forest on over-sampling data
from sklearn.ensemble import RandomForestClassifier

rf_over = RandomForestClassifier(n_estimators=100)
rf_over.fit(X_train_over,y_train_over)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [25]:
from sklearn.metrics import accuracy_score
y_pred_over = rf_over.predict(X_test_over)
print('over-sampling test accuracy: %.2f%%' % (accuracy_score(y_test_over, y_pred_over)*100))

over-sampling test accuracy: 98.04%


In [26]:
## random forest model with CV on over-sampling data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

rfcv_over = RandomForestClassifier(n_estimators=100, random_state=123)

accs_over = cross_val_score(estimator=rfcv_over, X=X_train_over, y=y_train_over, cv=5)
print(accs_over)
print(accs_over.mean())

[0.97916667 0.98958333 0.97916667 1.         0.97849462]
0.985282258064516
